# How to use `locals`
`locals` is a pure Python package that ingests JWST Wide-Field Slitless Spectroscopy data and returns a source catalog of all the low-mass stars in the field along with their calculated fundamental and secondary parameters.

### Requirements
- pip install SEDkit
- pip install bokeh

In [34]:
# Imports
from locals import source, catalog
from SEDkit import sed, spectrum, synphot
import astropy.units as q
import astropy.table as at
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

## Making an SED for a single source (no WFSS spectra)

For each spectrum in our WFSS data products, a `Source` object is created. `locals` then searches Vizier to find ancillary photometry, parallaxes, and spectral types. Then it assembles an SED from the available data and calculates fundamental and atmospheric parameters.

Let's pretend one of the brown dwarfs in our field is [LHS2924](http://simbad.u-strasbg.fr/simbad/sim-id?Ident=LHS2924&NbIdent=1&Radius=2&Radius.unit=arcmin&submit=submit+id). Here is what `locals` is doing for each source under the hood.

In [35]:
# Coordinates of LHS 2924 (Usually taken from the WFSS source_list)
ra = 217.180137*q.deg
dec = 33.177539*q.deg

# Make the source object
src = source.Source(ra=ra, dec=dec, name='LHS 2924', spectral_type='M9', radius=(1.06*q.R_jup,0.05*q.R_jup))

# Find some photometry
src.find_SDSS()
src.find_2MASS()
src.find_WISE()
src.find_PanSTARRS()

# Find a distance from Gaia DR2
src.find_Gaia()

# Take a look
print('\n',src.photometry,'\n')

Setting radius to (<Quantity 1.06 jupiterRad>, <Quantity 0.05 jupiterRad>)
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)

   band           eff         app_magnitude ...      abs_flux_unc      bandpass
                  um                       ... erg / (Angstrom cm2 s)         
-------- ------------------- ------------- ... ---------------------- --------
  SDSS.u 0.35565239687948447        21.945 ... 1.2549111006613915e-18   SDSS.u
  SDSS.g  0.4702495279331733        20.956 ...  8.182773370880929e-19   SDSS.g
   PS1.g 0.48491168757054565        20.567 ... 1.3484427632650426e-18    PS1.g
  SDSS.r  0.6175578881059804        18.582 ...  9.186342638736616e-19   SDSS.r
   PS1.r  0.6201185707505671        18.488 ...   6.60097046064012e-19    PS1.r
  Gaia.G  0.6597013110557894       16.6299 ...                    0.0   Gaia.G
  SDSS.i  0.7489976846298569        16.207 ...  2.544563527396815e-18   SDSS.i
   PS1.i  0.7534955578987176         16.21 ...  1.992420838499488e-18    PS1.

Now we can calculate the fundamental parameters and plot the results.

In [36]:
src.results
# 2275K from Fili15

param,value,unc,units
object,object,object,object
name,LHS 2924,--,--
age,6.0,4.0,Gyr
distance,10.99,0.02,pc
parallax,90.9962,0.12710000574588776,mas
radius,1.06,0.05,jupiterRad
SpT,M9,--,--
spectral_type,69.0,0.5,--
membership,--,--,--
fbol,7.56e-11,7.31e-13,erg / (cm2 s)


Let's fit a blackbody for fun.

In [37]:
src.fit_blackbody()


Blackbody fit: 1832 K


In [38]:
fig = src.plot(integral=True)

## Make SEDs for all sources in the WFSS field
Now that we see the output of a single source, we can run this on the entire `SourceCatalog` generated from the WFSS output. The difference here is that a WFSS spectrum will be included in each `Source`.

In [51]:
# Path to the JWST pipeline output directory
cat_path = resource_filename('locals', 'data/fake/')

# Calculate all sources in the pipeline output diretory
cat = catalog.SourceCatalog(cat_path, color_cut='brown_dwarfs')

No uncertainty array for this spectrum. Using SNR= 10
No uncertainty array for this spectrum. Using SNR= 10
No uncertainty array for this spectrum. Using SNR= 10
No uncertainty array for this spectrum. Using SNR= 10
No uncertainty array for this spectrum. Using SNR= 10
5/22 sources added to catalog after applying 'brown_dwarfs' color cuts


All the sources are stored as `Source` instances within the `SourceCatalog.sources` attribute, just as the example above.

In [52]:
# Get a source instance
source1 = cat.get_SED(0)

# Calculate the results
print(source1.results)

# Plot it
fig = source1.plot(spectra='all')

    param            value                unc              units    
------------- ------------------- -------------------- -------------
         name            Source 5                   --            --
          age                 6.0                  4.0           Gyr
     distance                  --                   --            --
     parallax                  --                   --            --
       radius 0.08134993688656778 0.017798157224262923        solRad
          SpT                L0.0                   --            --
spectral_type                70.0                  0.5            --
   membership                  --                   --            --
         fbol            3.01e-13             2.33e-15 erg / (cm2 s)
         mbol              19.821                0.008            --
         Lbol                  --                   --            --
     Lbol_sun                  --                   --            --
         Mbol                  -- 

In [53]:
source1.fit_spectral_type()

Hooray! Let's take a look at another one.

In [54]:
# Get the source instance
source2 = cat.get_SED(1)

# Calculate the results
print(source2.results)

# Plot it
fig = source2.plot(spectra='all')

    param            value                unc              units    
------------- ------------------- -------------------- -------------
         name            Source 7                   --            --
          age                 6.0                  4.0           Gyr
     distance                  --                   --            --
     parallax                  --                   --            --
       radius 0.08134993688656778 0.017798157224262923        solRad
          SpT                L0.0                   --            --
spectral_type                70.0                  0.5            --
   membership                  --                   --            --
         fbol            2.21e-13             1.53e-15 erg / (cm2 s)
         mbol              20.158                0.008            --
         Lbol                  --                   --            --
     Lbol_sun                  --                   --            --
         Mbol                  -- 

Fantastic!

Let's see a table of the results and plot all the SEDs!

In [55]:
cat.results[['name','spectral_type','fbol','radius','Teff_bb']]

name,spectral_type,fbol,radius,Teff_bb
,,erg / (cm2 s),solRad,K
object,object,object,object,object
Source 5,70.0,3.011234978903086e-13,0.08134993688656778,3012
Source 7,70.0,2.2076057120450164e-13,0.08134993688656778,2624
Source 11,70.0,1.297119660875548e-12,0.08134993688656778,3599
Source 16,70.0,3.181633971545836e-11,0.08134993688656778,3053
Source 17,70.0,1.6200642838010945e-13,0.08134993688656778,2986


In [50]:
# We imposed a color cut of NIRISS.F150W-NIRISS.F200W in (0.3, 0.5) and NIRISS.F158M-NIRISS.F150W in (-0.2, -0.1)
# Let's see if it worked
color = cat.plot('NIRISS.F150W-NIRISS.F200W', 'NIRISS.F158M-NIRISS.F150W')
show(color)

Nice!